# https://datahub.io/core/airport-codes#python

In [12]:
# Do all imports and installs here
from pyspark.sql.functions import udf
from pyspark.sql.session import SparkSession
import datetime
import pandas as pd
import pyspark.sql.functions as F
import psycopg2


In [9]:
data_quality_tables = ['fct_cic']
data_quality_fk_tables = ['dim_addr','dim_airline','dim_cnty','dim_port']
data_quality_fk_columns = ['i94addr','airline','i94cit','i94port']

for table in data_quality_tables:
    print(table)
    for idx, fk_table in enumerate(data_quality_fk_tables):
        print(idx, fk_table)
        fk_column = data_quality_fk_columns[idx]
        print(fk_column)



fct_cic
0 dim_addr
i94addr
1 dim_airline
airline
2 dim_cnty
i94cit
3 dim_port
i94port


In [13]:
spark = SparkSession.builder\
                    .config("spark.jars.packages","org.apache.hadoop:hadoop-aws:2.7.0")\
                    .appName("Project: i94")\
                    .getOrCreate()

In [14]:
# Read csv file(s)
csv_airport_codes = pd.read_csv('D:/Capstone-Project/Project-Workspace/airport-codes_csv.csv', quotechar='"')

In [15]:
csv_airport_codes.head()

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,icao_designator,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11.0,NaN,US,US-PA,Bensalem,00A,NaN,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NaN,US,US-KS,Leoti,00AA,NaN,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450.0,NaN,US,US-AK,Anchor Point,00AK,NaN,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820.0,NaN,US,US-AL,Harvest,00AL,NaN,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237.0,NaN,US,US-AR,Newport,NaN,NaN,NaN,"-91.254898, 35.6087"


In [16]:
csv_airport_codes = csv_airport_codes.dropna()

In [17]:
csv_airport_codes.head()

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,icao_designator,local_code,coordinates
223,03N,small_airport,Utirik Airport,4.0,OC,MH,MH-UTI,Utirik Island,K03N,UTK,03N,"169.852005, 11.222"
10384,ADC,small_airport,Andakombe Airport,3600.0,OC,PG,PG-EHG,Andekombe,AYAN,ADC,ADK,"145.744722222, -7.13722222222"
10414,AEK,small_airport,Aseki Airport,4106.0,OC,PG,PG-MPL,Aseki,AYAX,AEK,ASE,"146.19386673, -7.35080485552"
10474,AIE,small_airport,Aiome Airport,350.0,OC,PG,PG-MPM,Aiome,AYAO,AIE,AIO,"144.7307003, -5.145699978"
10475,AIH,small_airport,Aiambak Airport,90.0,OC,PG,PG-WPD,Aiambak,AYAK,AIH,AMB,"141.2675, -7.342777777779999"


In [18]:
sp_airport_codes = spark.createDataFrame(csv_airport_codes)

In [19]:
sp_airport_codes.printSchema()

root
 |-- ident: string (nullable = true)
 |-- type: string (nullable = true)
 |-- name: string (nullable = true)
 |-- elevation_ft: double (nullable = true)
 |-- continent: string (nullable = true)
 |-- iso_country: string (nullable = true)
 |-- iso_region: string (nullable = true)
 |-- municipality: string (nullable = true)
 |-- gps_code: string (nullable = true)
 |-- icao_designator: string (nullable = true)
 |-- local_code: string (nullable = true)
 |-- coordinates: string (nullable = true)



In [20]:
sp_airport_codes.coalesce(1).write.format('json').mode('overwrite').save('D:/capstone/solarhenge/dim_airport_codes')
